The Belgian Federal Parliament (De Kamer) provides an overview of all written questions of members of parliament to the ministers. There is no API. The questions and answers are available through pdfs or html pages. 

# Setting up

In [1]:
# show all outputs of cell, not merely of last line (i.e. default of Jupyter Notebook)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import os
import requests
from bs4 import BeautifulSoup

import pandas as pd

import re
import pickle

# Extract various Bulletins

First we create a function to obtain the urls of all so-called Bulletins in which the Federal Parliament provides the individual questions. 

In [3]:
def scrape_list_bulletins(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        bulletin_urls = []
    
        for link in soup.find_all('a', href=True):
            if link['href'].startswith('showpage.cfm?&language=nl&cfm=/site/wwwcfm/qrva/qrvatoc.cfm?legislat'):
                bulletin_url = f"https://www.dekamer.be/kvvcr/{link['href']}"

                # print(bulletin_url)

                bulletin_urls.append(bulletin_url)

        # Remove duplicates, since urls are 2 times shown
        bulletin_urls = sorted(set(bulletin_urls))
        
        return bulletin_urls

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

In [4]:
# Define url of page with overview of urls of bulletins
url_main_page = "https://www.dekamer.be/kvvcr/showpage.cfm?section=/qrva&language=nl&cfm=qrvaList.cfm"

# Obtain urls
bulletin_urls_main_page = scrape_list_bulletins(url_main_page)

# Inspect results
bulletin_urls_main_page[:5]
len(bulletin_urls_main_page)

['https://www.dekamer.be/kvvcr/showpage.cfm?&language=nl&cfm=/site/wwwcfm/qrva/qrvatoc.cfm?legislat=55&bulletin=B001',
 'https://www.dekamer.be/kvvcr/showpage.cfm?&language=nl&cfm=/site/wwwcfm/qrva/qrvatoc.cfm?legislat=55&bulletin=B002',
 'https://www.dekamer.be/kvvcr/showpage.cfm?&language=nl&cfm=/site/wwwcfm/qrva/qrvatoc.cfm?legislat=55&bulletin=B003',
 'https://www.dekamer.be/kvvcr/showpage.cfm?&language=nl&cfm=/site/wwwcfm/qrva/qrvatoc.cfm?legislat=55&bulletin=B004',
 'https://www.dekamer.be/kvvcr/showpage.cfm?&language=nl&cfm=/site/wwwcfm/qrva/qrvatoc.cfm?legislat=55&bulletin=B005']

125

# Extract relevant information for each question

Each Bulletin contains various questions. The html page of the Bulletin provides for each question some information, which can extract:
* ID (i.e. the unique identifier of each question)
* Auteur (i.e. author, the member of parliament posing the question)
* Partij (i.e. the party of which the author is a member)
* Departement (i.e. the minister to whom the question is directed)
* Titel (i.e. subject of the question)
* Datum ingediend (i.e. the date when the question was asked)
* Antwoord gepubliceerd (i.e. the hyperlink to the answer to the question)

To obtain these elements, we create various functions. 

The html structure is not so clear (i.e. not all relevant elements are contained within a single container, so using specific anchors does not suffice). IN particular the date the question was asked and whether an answer was provided before publishing the question or not is provided in a different tag (see example below). As indicated below, the `div` element includes most information, but not the date the question was asked and the hyperlink to the answer.

```html
<tr>
	<td valign="middle"><img src="/images/arrow3.png"></td><td><div class="linklist_1">
	232. 
	<br>
	
	<A HREF="showpage.cfm?section=qrva&language=nl&cfm=qrvaXml.cfm?legislat=55&dossierID=55-b101-1320-1814-2022202317928.xml">B101</A>
	</div></td>
	<td><div class="linklist_1">	
		<table width="100%">
        <tr><td class="txt" width="150">
		<i>Auteur</i></td><td class="txt">Dominiek
      Sneppe,
      VB (O1330)</td></tr>
		<tr><td class="txt">
		<i>Departement</i></td><td class="txt">Vice-eersteminister en Minister van Sociale Zaken en Volksgezondheid</td></tr>
		<tr><td class="txt"><i>Titel</i></td><td class="txt">Privéziekenhuizen.</div></td></tr>

 

		
		
				
		<tr>
		<td class="txt"><i>Datum indiening</i></td>
		<td class="txt">06/12/2022&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
		<i>
		Vraag, antwoord ontvangen :&nbsp;
					
						<A HREF="http://www.dekamer.be/QRVA/pdf/55/55K0101.pdf" target="_blank">B101</A>
					
		</i>
		</td></tr>

		
		
		
		

		<tr><td class="txt"><i>Trefwoord</i></td><td class="txt"></td></tr>
		</table>
	</td></tr>
 ```

A first helper function extracts the relevant textual elements from the hltm strings. This approach is less robust then using BeautifulSoup to extract html elements. However, the latter approach is more error prone in this context, possibly due to the overall set up of the html page (see earlier). We will later on use this function to extract the relevant parts of the html code for each segment relating to a specific question.

In [5]:
def extract_text(s, start, end):
    start_index = s.find(start)
    if start_index != -1:
        start_index += len(start)
        end_index = s.find(end, start_index)
        if end_index != -1:
            return s[start_index:end_index].strip()

The second helper function is a function to split the information obtained through the tag `Auteur`. This does not only contain the actual author of the question, but also its party and an ID of the question. 

In [6]:
# # Create helper function 
# def split_author(input_string):
#     """
#     Function to split strings as obtained from html page into name of member, his/her party and id of question

#     e.g. 
#     'Anneleen\n      Van Bossuyt,\n      N-VA (07354)'
#     ('Anneleen Van Bossuyt', ' N-VA ', '07354')
#     """
#     if input_string:
#         # Remove unnecessary characters (newlines and bracket at end (of id number))
#         cleaned_string = input_string.replace("\n", "").rstrip(")")
        
#         # Replace any sequence of spaces with a single space
#         cleaned_string = re.sub(r'\s+', ' ', cleaned_string)
    
#         # split on comma (between name and party) and left bracket (between party and id number)
#         name, party, id_number = re.split(r',|\(', cleaned_string)
    
#         return name, party, id_number
#     else:
#         return None, None, None

In [7]:
# Create helper function 
def split_author(input_string):
    """
    Function to split strings as obtained from html page into name of member, his/her party and id of question

    e.g. 
    'Anneleen\n      Van Bossuyt,\n      N-VA (07354)'
    ('Anneleen Van Bossuyt', ' N-VA ', '07354')
    """
    # Remove unnecessary characters (newlines and bracket at end (of id number))
    cleaned_string = input_string.replace("\n", "").rstrip(")")
    
    # Replace any sequence of spaces with a single space
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string)

    # split on comma (between name and party) and left bracket (between party and id number)
    name, party, id_number = re.split(r',|\(', cleaned_string)

    return name, party, id_number

Then we create a function to actually scrape each bulletin and obtain the relevant information for each question, using the url of the specific bulletin page as input.

In [8]:
def scrape_bulletin(url):
    # Send an HTTP GET request to the URL
    response = requests.get(url)

    # If no error, proceed
    if response.status_code == 200:
        # Get the HTML content from the response and parse it
        soup = BeautifulSoup(response.content, 'html.parser')

        # Split soup string at every staring point of a relevant htlml block, 
        # i.e. indicated by "<div class="linklist_0">" or "<div class="linklist_1">"
        # There are separate div elements for the enumeration (e.g. 11., 25.).
        # we can avoid selecting thise too by adding the start of the html code in the relevant section,
        # i.e. "\n<table width="100%">\n"
        questions = re.split(r'<div class=\"linklist_[01]\">\n<table width="100%">\n', str(soup))
        
        # Create empty lis to store details
        details_question_list =[]

        # Iterate over all questions (i.e. the html strings of the relevant htlm blocks)
        # Do not iterate over the first element, since this is the html code up to the first relevant section 
        # (i.e. before the content relating to the questions)
        for question in questions[1:]: 
            # Extracting values for the specified tags
            author_full = extract_text(question, '<i>Auteur</i></td><td class="txt">', '</td>')
            department = extract_text(question, '<i>Departement</i></td><td class="txt">', '</td>')
            title = extract_text(question, '<i>Titel</i></td><td class="txt">', '</td>')
            date_submission_full= extract_text(question, '<i>Datum indiening</i></td>\n<td class="txt">', '</td>')
            date_submission = date_submission_full.split()[0] if date_submission_full else None
            answer_published = extract_text(question, '<a href="', '" target="_blank">')
        
            # Split string on about using dedicated function
            author, party_author, id_question = split_author(author_full)
        
            #Store elements in overall list
            details_question_list.append([id_question, author, party_author, department, 
                                          title, date_submission, answer_published])

        return details_question_list
                
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

In [9]:
# # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# # limit amounf of pages for testing
# bulletin_urls_main_page = bulletin_urls_main_page[:3]
# # +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

Then we apply the scraping function to all the bulletin pages. 

In [ ]:
# Initialize list with details on all questions
questions_all = []
# iterate over all urls of bulletin
for index, url in enumerate(bulletin_urls_main_page):
    # Obtain information of all questions in relevant bulletin
    questions_per_bulletin = scrape_bulletin(url)
    questions_all.extend(questions_per_bulletin) # extend to overall list (no appending to avoid nesting)
    print(f"Bulletins with questions processed: {index + 1}.") # Show progress

print("***** All bulletins processed. *****")

Bulletins with questions processed: 1.
Bulletins with questions processed: 2.
Bulletins with questions processed: 3.
Bulletins with questions processed: 4.
Bulletins with questions processed: 5.
Bulletins with questions processed: 6.
Bulletins with questions processed: 7.


This results in a list of the relevant elements per question. We turn this into a dataframe for easier handling and inspection.

In [ ]:
# Turn list into dataframe
questions_df = pd.DataFrame(questions_all,
                            columns = ["ID vraag", "Parlementslid", "Partij parlementslid", 
                                       "Minister (bevoegdheden)", "Onderwerp",
                                       "Datum ingediend", "Url publicatie"]
                           )

In [ ]:
# # Inspect results
questions_df.head()
questions_df["Partij parlementslid"].value_counts()
questions_df.dtypes

# Filter rows where at least 1 column has value 'None'
questions_df[questions_df.isna().any(axis=1)]

# Clean up resulting data

Upon inspection of the results, it seems some party names are incorrect. 
* Sometimes abbreviations are use (`VB` instead of `Vlaams Belang`).
* Or a party changed names during the legislative periode (e.g. `sp.a` v. `Vooruit` or `cdh` v. `Les Engagés`).
* Also, during the string extraction, their appear some issues with extracting `CD&V` (i.e. resulting in `CD&amp;V` instead).

In [ ]:
# Strip leading and trailing whitespace from column names
questions_df.columns = questions_df.columns.str.strip()

# Replace party names, accounting for trailing spaces
questions_df["Partij parlementslid"] = questions_df["Partij parlementslid"].str.strip().replace(['sp.a', 'Voorui'], 'Vooruit')
questions_df["Partij parlementslid"] = questions_df["Partij parlementslid"].str.strip().replace(['cdH', 'LENGAG'], 'Les Engagés')
questions_df["Partij parlementslid"] = questions_df["Partij parlementslid"].str.strip().replace('CD&amp;V', 'cd&v')

# Additional replacements, accounting for trailing spaces
questions_df["Partij parlementslid"] = questions_df["Partij parlementslid"].str.strip().replace({
    # ['sp.a', 'Voorui']: 'Vooruit',
    # ['cdH', 'LENGAG']: 'Les Engagés',
    # 'CD&amp;V': 'cd&v',
    'DéFI': 'Défi', 
    'VB': 'Vlaams Belang', 
    'INDEP': 'Onafhankelijk',
})

# Inspect restuls
questions_df["Partij parlementslid"].value_counts()

Furthermore, the subject generally contains a trailing dot, so we remove this. 

In [ ]:
questions_df["Onderwerp"] = questions_df["Onderwerp"] .str.rstrip('.')

Also, the date of the questions is in string format. Hence, we recast to datetime format.

In [ ]:
# Convert the 'date_column' from string to datetime format
questions_df["Datum ingediend"] = pd.to_datetime(questions_df["Datum ingediend"], 
                                                dayfirst=True)

# # Extract only the date part from the datetime column
# questions_df['Datum ingediend'] = questions_df['Datum ingediend'].dt.date

# Inspect results
questions_df.dtypes
questions_df.head()

questions_df["Datum ingediend"].min()
questions_df["Datum ingediend"].max()

## Add minister name, based on relevant timeframe holding specific position

Also, the responsible minister is merely name by the competences he/she manages. It seems more user-friendly to replace this by the actual name of the minister. However, some positions were held by multiple people, consequtively. Hence, the positions cannot serve as a unique key. For clarity, we first identify all relevant positions. Later on, we handle this ambiguity. 

In [ ]:
minister_competences_2_names_dict = {
    # Function allocated to multiple persons, unclear to distinguish based on allocated competences
    'Eerste Minister': 'Charles Michel', # during Michel II - 9 december 2018 - 27 oktober 2019
    'Eerste Minister': 'Sophie Wilmès', # 27 oktober 2019 - 30 november 2019 (before exit Reynders) 
    'Eerste Minister': 'Alexander De Croo', # 1 oktober 2020 - now

    # Michel II (9 december 2018 - 27 oktober 2019) (and not continued as such in Wilmès-I and / Wilmès-II)
        # See https://nl.wikipedia.org/wiki/Regering-Michel_II
    
    'Minister van Begroting en van Ambtenarenzaken, belast met de Nationale Loterij en Wetenschapsbeleid': 'Sophie Wilmès',
    'Minister van Werk, Economie en Consumenten, belast met Buitenlandse Handel, Armoedebestrijding, Gelijke Kansen en Personen met een beperking': 'Wouter Beke', # 2 juli 2019 - 2 oktober 2019 
    
    
    # Wilmès-I (27 oktober 2019 - 17 maart 2020)
        # See https://nl.wikipedia.org/wiki/Regering-Wilm%C3%A8s_I

        # Before exit Didier Reynders (27 oktober 2019 - 30 november 2019)
    'Vice-eersteminister en Minister van Buitenlandse en Europese Zaken, en van Defensie, belast met Beliris en de Federale Culturele Instellingen': 'Didier Reynders', 
    'Minister van Begroting en van Ambtenarenzaken, belast met de Nationale Loterij en Wetenschapsbeleid': 'David Clarinval', 
    'Vice-eersteminister en Minister van Justitie, belast met de Regie der Gebouwen': 'Koen Geens', 

        # After exit Didier Reynders: 
        # competences shifted to Wilmès, Clarinval and Geens (but remained same during Wilmès I and Wilmès II)
    
    # Wilmès-II (17 maart 2020 - 1 oktober 2020)
        # Zie https://nl.wikipedia.org/wiki/Regering-Wilm%C3%A8s_II


    # Wilmès I en Wilmès II (so no changes at 17 maart 2020 - so from 27 oktober 2019 or 30 november 2019 until 1 oktober 2020)
        # After exit Reynders (as of 30 november 2019 until 1 oktober 2020)
    'Eerste Minister, belast met Beliris en de Federale Culturele Instellingen': 'Sophie Wilmès', 
    'Vice-eersteminister en Minister van Begroting en van Ambtenarenzaken, belast met de Nationale Loterij en Wetenschapsbeleid': 'David Clarinval', 
    'Vice-eersteminister en Minister van Justitie, belast met de Regie der Gebouwen, en Minister van Europese Zaken': 'Koen Geens',

        # General (as of 27 oktober 2019 until 1 oktober 2020)
    'Vice-eersteminister en Minister van Financiën, belast met Bestrijding van de fiscale fraude, en Minister van Ontwikkelingszaken': 'Alexander De Croo',

    'Minister van Buitenlandse Zaken, en van Defensie': 'Philippe Goffin',
    'Minister van Digitale Agenda, Telecommunicatie en Post, belast met Administratieve Vereenvoudiging, Bestrijding van de sociale fraude, Privacy en Noordzee': 'Philippe De Backer',
    'Minister van Energie, Leefmilieu en Duurzame Ontwikkeling': 'Marie-Christine Marghem',
    "Minister van Middenstand, Zelfstandigen, Kmo's, Landbouw, en Maatschappelijke Integratie, belast met Grote Steden": 'Denis Ducarme',
    'Minister van Mobiliteit, belast met Belgocontrol en de Nationale Maatschappij der Belgische spoorwegen': 'François Bellot',
    'Minister van Pensioenen': 'Daniel Bacquelaine',
    'Minister van Sociale Zaken en Volksgezondheid, en van Asiel en Migratie': 'Maggie De Block',
    'Minister van Veiligheid en Binnenlandse Zaken': 'Pieter De Crem',
    'Minister van Werk, Economie en Consumenten, belast met Armoedebestrijding, Gelijke Kansen en Personen met een beperking': 'Nathalie Muylle', # 2 oktober 2019 - 27 oktober 2019 
     
    # Regering De Croo

    'Vice-eersteminister en Minister van Economie en Werk': 'Pierre-Yves Dermagne',
    'Vice-eersteminister en Minister van Buitenlandse Zaken, Europese Zaken en Buitenlandse Handel, en de Federale Culturele Instellingen': 'Sophie Wilmès', # 1 oktober - 14 juli 2022
    'Vice-eersteminister en Minister van Mobiliteit': 'Georges Gilkinet',
    'Vice-eersteminister en Minister van Financiën, belast met de Coördinatie van de fraudebestrijding': 'Vincent Van Peteghem',
    'Vice-eersteminister en Minister van Sociale Zaken en Volksgezondheid': 'Frank Vandenbroucke',
    'Vice-eersteminister en Minister van Ambtenarenzaken, Overheidsbedrijven, Telecommunicatie en Post': 'Petra De Sutter',
    'Vice-eersteminister en Minister van Justitie, belast met de Noordzee': 'Vincent Van Quickenborne', # 1 oktober 2020 - 22 oktober 2023
    'Vice-eersteminister en Minister van Justitie, belast met de Noordzee': 'Paul Van Tigchelt', # 22 oktober 2023 - now
    
    

    "Minister van Middenstand, Zelfstandigen, Kmo's en Landbouw, Institutionele Hervormingen en Democratische Vernieuwing": 'David Clarinval',
    "Minister van Middenstand, Zelfstandigen, Kmo's en Landbouw, Institutionele Hervormingen en Democratische Vernieuwing, belast met Buitenlandse Handel" :'David Clarinval', 
    'Minister van Pensioenen en Maatschappelijke Integratie, belast met Personen met een beperking, Armoedebestrijding en Beliris': 'Karine Lalieux',
    'Minister van Defensie': 'Ludivine Dedonder',
    'Minister van Klimaat, Leefmilieu, Duurzame Ontwikkeling en Green Deal': 'Zakia Khattabi',
    'Minister van Binnenlandse Zaken, Institutionele Hervormingen en Democratische Vernieuwing': 'Annelies Verlinden',
    'Minister van Ontwikkelingssamenwerking en Grootstedenbeleid': 'Meryame Kitir', # 1 oktober 2020 - 17 december 2022 
    'Minister van Ontwikkelingssamenwerking en Grootstedenbeleid': 'Caroline Gennez', # 17 december 2022 - now
    # 'Minister van Ontwikkelingssamenwerking, belast met Grote Steden' seems to be same as 'Minister van Ontwikkelingssamenwerking en Grootstedenbeleid'
    'Minister van Ontwikkelingssamenwerking, belast met Grote Steden': 'Meryame Kitir', # 1 oktober 2020 - 17 december 2022 
    'Minister van Ontwikkelingssamenwerking, belast met Grote Steden': 'Caroline Gennez', # 17 december 2022 - now
    'Minister van Energie': 'Tinne Van der Straeten',
    'Minister van Buitenlandse Zaken, Europese Zaken en Buitenlandse Handel, en de Federale Culturele Instellingen.': 'Hadja Lahbib', # 15 juli 2022 - now
    

    'Staatssecretaris voor Relance en Strategische Investeringen, belast met Wetenschapsbeleid, toegevoegd aan de Minister van Economie en Werk': 'Thomas Dermine',
    'Staatssecretaris voor Digitalisering, belast met Administratieve Vereenvoudiging, Privacy en de Regie der Gebouwen, toegevoegd aan de Eerste Minister': 'Mathieu Michel',
    'Staatssecretaris voor Digitalisering, belast met Administratieve Vereenvoudiging, Privacy en de Regie der Gebouwen, de Federale Culturele Instellingen, toegevoegd aan de Eerste Minister': 'Mathieu Michel',
    'Staatssecretaris voor Gendergelijkheid, Gelijke Kansen en Diversiteit, toegevoegd aan de Minister van Mobiliteit': 'Sarah Schlitz', # 1 oktober 2020 - 26 april 2023
    'Staatssecretaris voor Gendergelijkheid, Gelijke Kansen en Diversiteit, toegevoegd aan de Minister van Mobiliteit': 'Marie-Colline Leroy', # 2 mei 2023 - now
    'Staatssecretaris voor Asiel en Migratie, belast met de Nationale Loterij, toegevoegd aan de Minister van Binnenlandse Zaken, Institutionele Hervormingen en Democratische Vernieuwing': 'Sammy Mahdi', # 1 oktober 2020 - 28 juni 2022
    'Staatssecretaris voor Asiel en Migratie, toegevoegd aan de Minister van Binnenlandse Zaken, Institutionele Hervormingen en Democratische Vernieuwing': 'Nicole de Moor', # 28 juni 2022 - now # 'Nationale Loterij' now at Van Peteghem
    'Staatssecretaris voor Begroting en Consumentenbescherming, toegevoegd aan de Minister van Justitie, belast met de Noordzee': 'Eva De Bleeker', # 1 oktober 2020 - 18 november 2022 
    'Staatssecretaris voor Begroting en Consumentenbescherming, toegevoegd aan de Minister van Justitie, belast met de Noordzee': 'Alexia Bertrand', # 18 november 2022 - now
}

In [ ]:
# Temporary modification of dict, until splitting by time is possible
minister_competences_2_names_dict["Eerste minister"] = 'Alexander De Croo / Sophie Wilmès / Charles Michel'
minister_competences_2_names_dict['Minister van Begroting en van Ambtenarenzaken, belast met de Nationale Loterij en Wetenschapsbeleid'] = 'Sophie Wilmès / David Clarinval'
minister_competences_2_names_dict['Vice-eersteminister en Minister van Justitie, belast met de Noordzee'] = 'Vincent Van Quickenborne / Paul Van Tigchelt'
minister_competences_2_names_dict['Minister van Ontwikkelingssamenwerking en Grootstedenbeleid'] = 'Meryame Kitir / Caroline Gennez'
minister_competences_2_names_dict['Minister van Ontwikkelingssamenwerking, belast met Grote Steden'] = 'Meryame Kitir / Caroline Gennez'
minister_competences_2_names_dict['Staatssecretaris voor Gendergelijkheid, Gelijke Kansen en Diversiteit, toegevoegd aan de Minister van Mobiliteit'] = 'Sarah Schlitz / Marie-Colline Leroy'
minister_competences_2_names_dict['Staatssecretaris voor Begroting en Consumentenbescherming, toegevoegd aan de Minister van Justitie, belast met de Noordzee'] = 'Eva De Bleeker / Alexia Bertrand'

In [ ]:
# Assess results
minister_competences_2_names_dict.keys()

minister_competences_2_names_dict["Eerste minister"]
minister_competences_2_names_dict['Minister van Begroting en van Ambtenarenzaken, belast met de Nationale Loterij en Wetenschapsbeleid']
minister_competences_2_names_dict['Vice-eersteminister en Minister van Justitie, belast met de Noordzee'] 
minister_competences_2_names_dict['Minister van Ontwikkelingssamenwerking en Grootstedenbeleid']
minister_competences_2_names_dict['Minister van Ontwikkelingssamenwerking, belast met Grote Steden']
minister_competences_2_names_dict['Staatssecretaris voor Gendergelijkheid, Gelijke Kansen en Diversiteit, toegevoegd aan de Minister van Mobiliteit'] 
minister_competences_2_names_dict['Staatssecretaris voor Begroting en Consumentenbescherming, toegevoegd aan de Minister van Justitie, belast met de Noordzee']

In [ ]:
## Save minister_competences_2_names_dict for later use
# 1. Save as pkl
with open('../data/minister_competences_2_names_dict.pkl', 'wb') as file:
    pickle.dump(minister_competences_2_names_dict, file)

First, we map the names of the ministers to the relevant positions, using the created dict. For the ambiguous positions, we initially use the grouped names as inserted above (e.g. 'Alexander De Croo / Sophie Wilmès / Charles Michel' for 'Eerste minister').

In [ ]:
# map names to positions using dict
questions_df["Minister"] = questions_df["Minister (bevoegdheden)"].map(minister_competences_2_names_dict)

# Inspect results
questions_df.head()
questions_df[questions_df["Minister"].isna()]
questions_df["Minister"].unique()

Then we perform detailed replacement for the relevant positions that were held by multiple people, based on the relevant time frame as defined above:
* Eerste minister
* Minister van Begroting en van Ambtenarenzaken, belast met de Nationale Loterij en Wetenschapsbeleid
* Vice-eersteminister en Minister van Justitie, belast met de Noordzee 
* Minister van Ontwikkelingssamenwerking en Grootstedenbeleid
* Minister van Ontwikkelingssamenwerking, belast met Grote Steden
* Staatssecretaris voor Gendergelijkheid, Gelijke Kansen en Diversiteit, toegevoegd aan de Minister van Mobiliteit
* Staatssecretaris voor Begroting en Consumentenbescherming, toegevoegd aan de Minister van Justitie, belast met de Noordzee

In [ ]:
# Modify 'Minister column' if its value matches 'Eerste minister' based on relevant time frames as indicated above, using the date column
    # during Michel II - 9 december 2018 - 27 oktober 2019
questions_df.loc[(
    questions_df["Datum ingediend"] < "2019-10-27") & (
        questions_df['Minister'] == 'Alexander De Croo / Sophie Wilmès / Charles Michel'), "Minister"] = 'Charles Michel' 
    # During Wilmès I - 27 oktober 2019 - 30 november 2019 (before exit Reynders) 
questions_df.loc[(
    questions_df["Datum ingediend"] >= "2019-10-27") & (
        questions_df["Datum ingediend"] < "2020-10-01") & (
        questions_df['Minister'] == 'Alexander De Croo / Sophie Wilmès / Charles Michel'), "Minister"] = 'Sophie Wilmès' 
    # 1 oktober 2020 - now
questions_df.loc[(
    questions_df["Datum ingediend"] >= "2020-09-30") & (
        questions_df['Minister'] == 'Alexander De Croo / Sophie Wilmès / Charles Michel'), "Minister"] = 'Alexander De Croo'

In [ ]:
# Modify 'Minister van Begroting en van Ambtenarenzaken, belast met de Nationale Loterij en Wetenschapsbeleid'
    # during Michel II (9 december 2018 - 27 oktober 2019) 
questions_df.loc[(
    questions_df["Datum ingediend"] < "2019-10-27") & (
        questions_df['Minister'] == 'Sophie Wilmès / David Clarinval'), "Minister"] = 'Sophie Wilmès' 

    # During Wilmès-I but before exit Didier Reynders (27 oktober 2019 - 30 november 2019)
questions_df.loc[(
    questions_df["Datum ingediend"] < "2019-10-27") & (
        questions_df['Minister'] == 'Sophie Wilmès / David Clarinval'), "Minister"] = 'David Clarinval' 

In [ ]:
# Modify 'Vice-eersteminister en Minister van Justitie, belast met de Noordzee'
    # before 22 oktober 2023
questions_df.loc[(
    questions_df["Datum ingediend"] < "2023-10-23") & (
        questions_df['Minister'] == 'Vincent Van Quickenborne / Paul Van Tigchelt'), "Minister"] = 'Vincent Van Quickenborne'

    # after 22 oktober 2023
questions_df.loc[(
    questions_df["Datum ingediend"] >= "2023-10-23") & (
        questions_df['Minister'] == 'Vincent Van Quickenborne / Paul Van Tigchelt'), "Minister"] = 'Paul Van Tigchelt'

In [ ]:
# Modify 'Minister van Ontwikkelingssamenwerking en Grootstedenbeleid'
    # before 17 december 2022
questions_df.loc[(
    questions_df["Datum ingediend"] < "2022-12-17") & (
        questions_df['Minister'] == 'Meryame Kitir / Caroline Gennez'), "Minister"] = 'Meryame Kitir'

    # after 17 december 2022
questions_df.loc[(
    questions_df["Datum ingediend"] >= "2022-12-17") & (
        questions_df['Minister'] == 'Meryame Kitir / Caroline Gennez'), "Minister"] = 'Caroline Gennez'

In [ ]:
# Modify 'Minister van Ontwikkelingssamenwerking, belast met Grote Steden'
    # before 17 december 2022
questions_df.loc[(
    questions_df["Datum ingediend"] < "2022-12-17") & (
        questions_df['Minister'] == 'Meryame Kitir / Caroline Gennez'), "Minister"] = 'Meryame Kitir'

    # after 17 december 2022
questions_df.loc[(
    questions_df["Datum ingediend"] >= "2022-12-17") & (
        questions_df['Minister'] == 'Meryame Kitir / Caroline Gennez'), "Minister"] = 'Caroline Gennez'

In [ ]:
# Modify 'Staatssecretaris voor Gendergelijkheid, Gelijke Kansen en Diversiteit, toegevoegd aan de Minister van Mobiliteit'
    # before 1 May 2023
questions_df.loc[(
    questions_df["Datum ingediend"] < "2023-05-01") & (
        questions_df['Minister'] == 'Sarah Schlitz / Marie-Colline Leroy'), "Minister"] = 'Sarah Schlitz'

    # after 26 april 2023
questions_df.loc[(
    questions_df["Datum ingediend"] >= "2023-05-01") & (
        questions_df['Minister'] == 'Sarah Schlitz / Marie-Colline Leroy'), "Minister"] = 'Marie-Colline Leroy'

In [ ]:
# Modify 'Staatssecretaris voor Begroting en Consumentenbescherming, toegevoegd aan de Minister van Justitie, belast met de Noordzee'
    # before 18 November 2022
questions_df.loc[(
    questions_df["Datum ingediend"] < "2022-11-18") & (
        questions_df['Minister'] == 'Eva De Bleeker / Alexia Bertrand'), "Minister"] = 'Eva De Bleeker'

    # after 26 april 2023
questions_df.loc[(
    questions_df["Datum ingediend"] >= "2022-11-18") & (
        questions_df['Minister'] == 'Eva De Bleeker / Alexia Bertrand'), "Minister"] = 'Alexia Bertrand'

In [ ]:
# Inspect results (include check to ensure that names assigned for all posts / competences)
questions_df.head()
questions_df[questions_df["Minister"].isna()]
questions_df["Minister"].unique()

## Combine column of subject and url to get clickable markdown link

In [ ]:
# Function to create markdown-style links
def create_markdown_link(row):
    return f"[{row['Onderwerp']}]({row['Url publicatie']})"

In [ ]:
# Apply the function to create a new column 'Onderwerp (url)'
questions_df["Onderwerp (url)"]= questions_df.apply(create_markdown_link, axis=1)

# # Drop original columns
# questions_df = questions_df.drop(['Onderwerp'], axis=1)
# questions_df = questions_df.drop(['Url publicatie'], axis=1)

In [ ]:
# Inspect results (include check to ensure that names assigned for all posts / competences)
questions_df.head()
questions_df.columns

# Save output

In [ ]:
## Save details_questions_term_df for later use
# 1. Save as pkl
with open('../data/federal_details_questions_df.pkl', 'wb') as file:
    pickle.dump(questions_df, file)

# 2. Save as csv
questions_df.to_csv(path_or_buf = '../data/federal_details_questions_df.csv',
                               sep = ";",
                               encoding = "utf-16", # to ensure trema's are well handled (e.g. Koen Daniëls)
                               index = False)